In [64]:
import pandas as pd
import numpy as np
from pydicom import dcmread, read_file
from ast import literal_eval
import skimage as sk
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import shutil

In [48]:
#crear lista de rutas para imagenes
rutas=open ('rutas.txt','w')
for dir_nom, _, arch_noms in os.walk('/home/jair/COVID/siimcovid/imagesdcm'):
    for arch_nom in arch_noms:
        rutas.write(os.path.join(dir_nom, arch_nom)+'\n')

rutas.close() 

In [ ]:
#Guardar info de interes sobre dataset
with open('rutas.txt', 'r') as archivo:
    # Itera sobre cada línea en el archivo
    #tamaño imagen
    tam=[]
    #modo monocrome
    mode=[]
    for ruta in tqdm(archivo):
        # Procesa la línea
        dicom=dcmread(ruta.replace('\n',''))
        mod=dicom.PhotometricInterpretation
        img=dicom.pixel_array
        tam.append(img.shape)
        mode.append(mod)
        
#guardar tamaños en csv
df = pd.DataFrame(tam, columns=['width', 'height'])
#Guardar DataFrame como CSV
df.to_csv('tams.csv', index=False)

#guardar modo en csv
df = pd.DataFrame(mod, columns=['mode'])
#Guardar DataFrame como CSV
df.to_csv('mod.csv', index=False)

In [38]:
info=pd.read_csv("SIIM/info.csv")
info.head()

,rutas,width,height,mode,label
0,cd5dd5e6f3f5/b2ee36aa2df5/d8ba599611e5.dcm,2336,2836,MONOCHROME1,Negative for Pneumonia
1,49358afcfb80/60a49211f5df/29b23a11d1e4.dcm,3488,4256,MONOCHROME2,Typical Appearance
2,e4b50e7402c3/59f646771321/8174f49500a5.dcm,2330,2846,MONOCHROME2,Negative for Pneumonia
3,e4b50e7402c3/d289a11b2e85/d54f6204b044.dcm,2330,2846,MONOCHROME2,Negative for Pneumonia
4,92aad2d01be8/60fe0c912619/d51cadde8626.dcm,3488,4256,MONOCHROME2,Typical Appearance


In [39]:
info_study=pd.read_csv("SIIM/train_study_level.csv")
info_study.head()

,id_study,label
0,00086460a852,Typical Appearance
1,000c9c05fd14,Atypical Appearance
2,00292f8c37bd,Negative for Pneumonia
3,005057b3f880,Negative for Pneumonia
4,0051d9b12e72,Atypical Appearance


In [40]:
info_img=pd.read_csv("SIIM/train_image_level.csv")
info_img.head()

,id,boxes,label,StudyInstanceUID
0,000a312787f2,"[[ 789.28836, 582.43035, 1026.65662, 1917.3...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f,[[]],[[]],ff0879eb20ed
2,0012ff7358bc,"[[ 677.42216, 197.97662, 867.79767, 999.782...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f,"[[ 2729, 2181.33331, 948.00012, 604]]",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891,"[[ 623.23328, 1050, 714, 1106], [ 2578.5666...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [41]:
Id=list(info_study['id_study'])
label=list(info_study['label'])

In [23]:
#Obten clase de cada imagen para agregar a la información del data set
clases=[]
for ruta in tqdm(info['rutas']):
    id_study=ruta.split('/')[0]
    if id_study in Id:
        indice = Id.index(id_study)
        clase=label[indice]
        clases.append(clase)
        
info['label']=clases
info.to_csv("SIIM/info.csv",index=False)

100%|█████████████████████████████████████| 6334/6334 [00:01<00:00, 5917.16it/s]


In [43]:
Id=list(info_img['id'])
bboxes=list(info_img['boxes'])

In [ ]:
#agregando bboxes a la informacion del conjunto de datos
bbs=[]
for ruta in tqdm(info['rutas']):
    id_img=ruta.split('/')[-1].replace('.dcm','')
    if id_img in Id:
        indice = Id.index(id_img)
        bb=bboxes[indice]
        bbs.append(bb)
info['boxes']=bbs
info.to_csv("SIIM/info.csv",index=False)        

100%|█████████████████████████████████████| 6334/6334 [00:01<00:00, 5432.78it/s]


In [60]:
info=pd.read_csv("info.csv")
info.head()

,rutas,width,height,mode,label,boxes
0,478ecae73c0f/ab1c8393562f/a6f2e4c387f0.dcm,2320,2832,MONOCHROME2,Atypical Appearance,"[[ 595.53935, 414.11282, 608.29559, 330.458..."
1,81711d215ec7/786be0021629/293dfd740406.dcm,2320,2832,MONOCHROME2,Atypical Appearance,"[[ 719.19628, 1348.36085, 468.92651, 648.06..."
2,55f9624fef0e/db0f28511428/41bbf4072fe2.dcm,2539,3050,MONOCHROME2,Atypical Appearance,"[[ 211.91156, 1458.23742, 1040.94028, 776.1..."
3,2c94d957e11e/e30e6ee21ab6/76bb17698bfa.dcm,3488,4256,MONOCHROME2,Atypical Appearance,"[[ 2560, 1740.8, 953.6001, 921.59985]]"
4,f2241ca2d0ff/32d0196a8616/cd588a3e1a88.dcm,3032,2520,MONOCHROME2,Atypical Appearance,"[[ 330.04627, 673.77778, 510.14603, 813.346..."


In [47]:
#redimensionando cada imagen y pasandolas a monocromo2 y png 1024
root='/home/jair/COVID/siimcovid/imagesdcm/'
rootd='/home/jair/COVID/siimcovid/images/1024/'
tam=(1024,1024)
for indice, fila in info.iterrows():
    r=root+fila['rutas'].split('/')[-1]
    dicom=dcmread(r)
    img=dicom.pixel_array
    imgr=sk.transform.resize(img, tam)
    if fila['mode']=='MONOCHROME1':
        imgr=np.amax(imgr)-imgr
    imgr=(np.maximum(imgr,0)/imgr.max())*255.0
    imgr=imgr.astype(np.uint8)
    sk.io.imsave(rootd+fila['rutas'].split('/')[-1].replace('dcm','png'), imgr)

/home/jair/anaconda3/envs/envj/lib/python3.11/site-packages/pydicom/pixel_data_handlers/numpy_handler.py:250: UserWarning: The length of the pixel data in the dataset (13262360 bytes) indicates it contains excess padding. 216296 bytes will be removed from the end of the data
  warnings.warn(msg)


In [66]:
rooto='/home/jair/COVID/siimcovid/images/1024/'
rootd='/home/jair/COVID/siimcovid/1024/'

for idd in tqdm(info['rutas']):
    ro=rooto+idd.split('/')[-1].replace('dcm','png')
    shutil.move(ro, rootd)

100%|████████████████████████████████████| 5778/5778 [00:00<00:00, 19512.33it/s]


In [13]:
import pandas as pd
import numpy as np
from pydicom import dcmread, read_file
from ast import literal_eval
import skimage as sk
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import shutil

def escala_bbox(bbox, widthO, heightO, resize):
    sy = resize / widthO
    sx = resize / heightO
    xr = bbox[0] * sx
    yr = bbox[1] * sy
    wr = bbox[2] * sx
    hr = bbox[3] * sy
    
    # Ajuste para asegurar que w y h no excedan los límites de la imagen
    if xr + wr > resize and resize - xr>0:
        wr = resize - xr-2
    if yr + hr > resize and resize - yr>0:
        hr = resize - yr-2
    
    return [xr, yr, wr, hr]

def xywh_yolo(bb,size):
    cx=bb[0]+(bb[2]/2)
    cy=bb[1]+(bb[3]/2)
    xc=cx/size
    yc=cy/size
    wc=bb[2]/size
    hc=bb[3]/size
    return [xc,yc,wc,hc]

def yolon_to_xywh(bb,size):
    xc=bb[0] * size
    yc=bb[1] * size
    wp=bb[2] * size
    hp=bb[3] * size
    x = xc - w/2
    y = yc - h/ 2
    return [x, y, wp, hp]

def xywh_xiyixfyf(bbox):
    x1=bbox[0]
    y1=bbox[1]
    x2=bbox[0]+bbox[2]
    y2=bbox[1]+bbox[3]
    return [x1,y1,x2,y2]

mode='validation'
ri='/home/jair/COVID/siimcovid/'+mode+'_ord.csv'
ro='/home/jair/COVID/siimcovid/images/640/'
rd='/home/jair/COVID/siimcovid/datasets/yolo/images/'+mode+'/'
anno={'Typical Appearance':0,'Negative for Pneumonia':1,'Indeterminate Appearance':2,'Atypical Appearance':3}
data=pd.read_csv(ri)
#creando dataset para YOLO las imagenes ya se redimensionaron por ende solo se redmensionan bboxes
for indice, fila in tqdm(data.iterrows()):
    if not os.path.exists(rd+fila['rutas'].split('/')[-1].replace('dcm','png')):
        r=ro+fila['rutas'].split('/')[-1].replace('dcm','png')   
        img=sk.io.imread(r)
        rgb = np.zeros((640,640,3))
        rgb[...,0] =img
        rgb[...,1] =img
        rgb[...,2] =img
        rgb=(rgb*-255).astype(np.uint8)
        sk.io.imsave(rd+fila['rutas'].split('/')[-1].replace('dcm','png'),rgb)
    if fila['label'] != 'Negative for Pneumonia':
        for bbox in literal_eval(fila['boxes']):
            bbr=escala_bbox(bbox,fila['width'],fila['height'],640)
            bby=xywh_yolo(bbr,640)
            
            with open(rd.replace('images','labels')+fila['rutas'].split('/')[-1].replace('dcm','txt'), 'a') as file:
                file.write(str(anno[fila['label']])+' '+str(bby[0])+' '+str(bby[1])+' '+str(bby[2])+' '+str(bby[3])+'\n')
    else:
        with open(rd.replace('images','labels')+fila['rutas'].split('/')[-1].replace('dcm','txt'), 'a') as file:
            file.write(str(anno[fila['label']])+' 0.500781 0.500781 0.9946 0.9946\n')
            

            

1155it [02:14,  8.59it/s]


In [16]:
import pandas as pd
import numpy as np
from pydicom import dcmread, read_file
from ast import literal_eval
import skimage as sk
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import shutil

def escala_bbox(bbox, widthO, heightO, resize):
    sy = resize / widthO
    sx = resize / heightO
    xr = bbox[0] * sx
    yr = bbox[1] * sy
    wr = bbox[2] * sx
    hr = bbox[3] * sy
    
    # Ajuste para asegurar que w y h no excedan los límites de la imagen
    if xr + wr > resize and resize - xr>0:
        wr = resize - xr-2
    if yr + hr > resize and resize - yr>0:
        hr = resize - yr-2
    
    return [xr, yr, wr, hr]

def xywh_yolo(bb,size):
    cx=bb[0]+bb[2]/2
    cy=bb[1]+bb[3]/2
    xc=cx/size
    yc=cy/size
    wc=bb[2]/size
    hc=bb[3]/size
    return [xc,yc,wc,hc]

def yolon_to_xywh(bb,size):
    x=bb[0]*size-bb[2]*size/2
    y=bb[1]*size-bb[3]*size/2
    wp=bb[2]*size
    hp=bb[3]*size
    return [x, y, wp, hp]

def xywh_xiyixfyf(bbox):
    x1=bbox[0]
    y1=bbox[1]
    x2=bbox[0]+bbox[2]
    y2=bbox[1]+bbox[3]
    return [x1,y1,x2,y2]

mode='validation'
ri='/home/jair/COVID/siimcovid/'+mode+'_ord.csv'
ro='/home/jair/COVID/siimcovid/images/640/'
rd='/home/jair/COVID/NIHX14/datasets/yolo/640/images/'+mode+'/'
anno={'Typical Appearance':0,'Negative for Pneumonia':1,'Indeterminate Appearance':2,'Atypical Appearance':3}
data=pd.read_csv(ri)

#creando dataset para Retiannet las imagenes ya se redimensionaron por ende solo se redmensionan bboxes

info = pd.DataFrame(columns=['path','x1','y1','x2','y2','class'])
pth=[]
x1=[]
y1=[]
x2=[]
y2=[]
c=[]
for indice, fila in tqdm(data.iterrows()):
    name=fila['rutas'].split('/')[-1].replace('dcm','png') 
    for bbox in literal_eval(fila['boxes']):
        pth.append(rd+name)
        if fila['label'] !='Negative for Pneumonia':
            bbr=escala_bbox(bbox,fila['width'],fila['height'],640)
            bbrf=xywh_xiyixfyf(bbr)
            x1.append(int(bbrf[0]))
            y1.append(int(bbrf[1]))
            x2.append(int(bbrf[2]))
            y2.append(int(bbrf[3]))
            c.append(fila['label'])
        else:
            x1.append(0)
            y1.append(0)
            x2.append(640)
            y2.append(640)
            c.append(fila['label'])      

info['path']=pth
info['x1']=x1
info['y1']=y1
info['x2']=x2
info['y2']=y2
info['class']=c

info=info.sample(frac=1).reset_index(drop=True)
info.to_csv('/home/jair/COVID/siimcovid/datasets/retinanet/'+mode+'.csv', index=False)

1155it [00:00, 5604.43it/s]
